In [51]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory, TerminationCondition
import psycopg2

## Conectar Base de datos y Preparacion de los datos a utilizar

In [52]:
conexion = psycopg2.connect(
        dbname="buap",
        user="postgres",
        password="contrasena",
        host="localhost",
        port="5432"
    )
cursor = conexion.cursor()
datos = {}

In [53]:
# Profesores (con nombre)
cursor.execute("SELECT id_profesor, nombre FROM Profesor LIMIT 50")
profesores = cursor.fetchall()
datos['Id_Profesores'] = [row[0] for row in profesores] # Lista de IDs de profesores
datos['nombres_profesor'] = {row[0]: row[1] for row in profesores} # Diccionario de nombres de profesores

In [54]:
# Materias (con nombre)
cursor.execute("SELECT id_materia, nombre, horas_por_semana, id_tipo_clase FROM Materia LIMIT 50")
materias = cursor.fetchall()
datos['id_Materias'] = [row[0] for row in materias] # Lista de IDs de materias
datos['nombres_materia'] = {row[0]: row[1] for row in materias} # Diccionario de nombres de materias
datos['Horas_Semana_Materia'] = {row[0]: row[2] for row in materias} # Diccionario de horas por semana de cada materia
datos['Tipo_Clase'] = {row[0]: row[3] for row in materias} # Diccionario de tipo de clase de cada materia

In [55]:
# Aulas
cursor.execute("SELECT id_aula, id_edificio, id_tipo_clase FROM Aula")
rows = cursor.fetchall()

datos['Aulas'] = [(row[0], row[1]) for row in rows] # Lista de IDs compuestos (id_aula, id_edificio)
datos['Tipo_Aula'] = {(row[0], row[1]): row[2] for row in rows}# Diccionario: clave compuesta (id_aula, id_edificio) -> tipo clase


# Bloques horarios
datos['Dia'] = list(range(1, 6)) # Lunes a Viernes
datos['Hora'] = list(range(1, 11)) # 10 bloques horarios por día/ de 8 am a 6 pm

In [56]:
aulas_param = datos['Aulas']  # lista de tuplas (id_aula, id_edificio)

query = """
SELECT A.id_aula, A.id_edificio, M.id_materia,
       CASE WHEN A.id_tipo_clase = M.id_tipo_clase THEN 1 ELSE 0 END AS compatible,
       M.id_tipo_clase AS tipo_materia,
       A.id_tipo_clase AS tipo_aula
FROM Aula A
CROSS JOIN Materia M
WHERE (A.id_aula, A.id_edificio) = ANY(%s) AND M.id_materia = ANY(%s)
"""

cursor.execute(query, (aulas_param, datos['id_Materias']))

datos['Compatibiliadad_Aula_Materia'] = {
    ((row[0], row[1]), row[2]): {
        'compatible': row[3],
        'tipo_materia': row[4],
        'tipo_aula': row[5]
    }
    for row in cursor.fetchall()
}


In [57]:
from collections import defaultdict
datos['Materias_por_Profesor'] = defaultdict(list)
datos['Profesores_por_Materia'] = defaultdict(list)
datos['Profesores_Imaginarios'] = []
datos['Profesores_Totales'] = []

cursor.execute("""
    SELECT id_profesor, id_materia 
    FROM profesor_materia
    WHERE id_profesor = ANY(%s) AND id_materia = ANY(%s)
""", (datos['Id_Profesores'], datos['id_Materias']))

for profe_id, materia_id in cursor.fetchall():
    datos['Materias_por_Profesor'][profe_id].append(materia_id)
    datos['Profesores_por_Materia'][materia_id].append(profe_id)

materias_sin_profesor = [m for m in datos['id_Materias'] if m not in datos['Profesores_por_Materia']]

profesores_imaginarios = [f"PA_{i}" for i in range(len(materias_sin_profesor))]
datos['Profesores_Imaginarios'] = profesores_imaginarios
print("Se han creado", len(profesores_imaginarios), "profesores imaginarios para las materias sin profesor asignado.")
#Asignación de profesores imaginarios
for i, materia_id in enumerate(materias_sin_profesor):
    profe_imag = profesores_imaginarios[i]
    datos['Profesores_por_Materia'][materia_id].append(profe_imag)
    datos['Materias_por_Profesor'][profe_imag].append(materia_id)

datos['Profesores_Totales'].extend(datos['Id_Profesores'])
datos['Profesores_Totales'].extend(profesores_imaginarios)

datos['nombres_profesor'].update({
    p: f"ProfFicticio_{i}" for i, p in enumerate(profesores_imaginarios)
})

Se han creado 29 profesores imaginarios para las materias sin profesor asignado.


In [58]:
datos['Materia_Sesion_Minima'] = {id_materia: 1 for id_materia in datos['id_Materias']}
datos['Materia_Sesion_Maxima'] = {id_materia: 3 for id_materia in datos['id_Materias']}
datos['Hora_Minima_Profesor'] = {id_prof: 0 for id_prof in datos['Profesores_Totales']}
datos['Hora_Maxima_Profesor'] = {id_prof: 20 for id_prof in datos['Profesores_Totales']}

## Crear Modelo de Pyomo

---

### Conjuntos

- $P: \text{Profesores}$
    - $P_R: \text{Profesores Reales}$
    - $P_I: \text{Profesores Imaginarios}$
    - $P = P_R \cup\ P_I$
- $P(j):$ Profesores que pueden enseñar la materia $j$
    - $P(j) \subset P$
- $M: \text{Materias}$
- $M(i):$ Materias que puede dar el profesor $i$
    - $M(i) \subset M$
- $A: \text{Aulas}$
    - $A = A_{salon}\ \cup\ A_{edificio}$
- $BH: \text{Bloques Horarios}$
    - $D: \text{Dias}, D = \{1, 2,\ \dots, 5\}$
    - $H: \text{Horarios}, H = \{1, 2,\ \dots, 10\}$
    - $BH = D \times H = \{(d, h)\ |\ d \in D, h \in H\}$
- $T: \text{Tipo}$
    - $T = \{ 1, 2, 3, 4 \}$

In [59]:
# Definir los conjuntos
model = pyo.ConcreteModel()

def M_prof_init(model, i):
    return datos['Materias_por_Profesor'][i]

def P_mat_init(model, j):
    return datos['Profesores_por_Materia'][j]

# Conjuntos
model.ProfesoresReales = pyo.Set(initialize=datos['Id_Profesores'])
model.ProfesoresImaginarios = pyo.Set(initialize=datos['Profesores_Imaginarios'])
model.Profesores = pyo.Set(initialize=datos['Profesores_Totales'])
model.Materias = pyo.Set(initialize=datos['id_Materias'])
model.M_prof = pyo.Set(model.Profesores, initialize=M_prof_init)
model.P_mat = pyo.Set(model.Materias, initialize=P_mat_init)
model.Aulas = pyo.Set(dimen=2, initialize=[(aula_id, edificio_id) for aula_id, edificio_id in datos['Aulas']])
model.BloquesHorarios = pyo.Set(dimen=2, initialize=[(dia, hora) for dia in datos['Dia'] for hora in datos['Hora']])
model.Dias = pyo.Set(initialize=datos['Dia'])
model.Horas = pyo.Set(initialize=datos['Hora'])
model.Tipo = pyo.RangeSet(1, 4)

### Parametros

- $\tau(j):$ Horas a la semana del materia $j$
- $\sigma (j):$ Tipo de clase de la materia $j$
- $\sigma (k):$ Tipo de aula del aula $k$
- $\omega (k, j) = \begin{cases} 1 & \text{si el aula}\ k\ \text{es compatible con la materia}\ j\\ 0 & \text{si no} \end{cases}$

- $ \omega (i, j) = \begin{cases} 1 & \text{si el profesor}\ i\ \text{puede dar la materia}\ j\\ 0 & \text{si no} \end{cases}$
- $\psi_{min}(j):$ Duración mínima de la sesión del curso $j$
- $\psi_{max}(j):$ Duración máxima de la sesión del curso $j$
- $\gamma_{min}(i):$ Horas mínimas que puede impartir el profesor $i$
- $\gamma_{max}(i):$ Horas máximas que puede impartir el profesor $i$

In [60]:
# Evitar sobreescribir
for attr in ['tau', 'sigma_materia', 'sigma_aula', 'omega_aula_materia', 'omega_prof_materia', 'Psi_Min', 'Psi_Max']:
    if hasattr(model, attr):
        model.del_component(getattr(model, attr))

model.tau = pyo.Param(model.Materias, initialize=datos['Horas_Semana_Materia'])
model.sigma_materia = pyo.Param(model.Materias, initialize=datos['Tipo_Clase'])
model.sigma_aula = pyo.Param(model.Aulas, initialize=datos['Tipo_Aula'])

model.omega_aula_materia = pyo.Param(
    model.Aulas, model.Materias,
    initialize={
        (aula, materia): datos['Compatibiliadad_Aula_Materia'].get((aula, materia), {}).get('compatible', 0)
        for aula in model.Aulas for materia in model.Materias
    }
)

model.omega_prof_materia = pyo.Param(
    model.Profesores, model.Materias,
    initialize={
        (profe_id, materia_id): 1 if materia_id in datos['Materias_por_Profesor'][profe_id] else 0
        for profe_id in model.Profesores for materia_id in model.Materias
    }
)

model.Psi_Min = pyo.Param(model.Materias, initialize=datos['Materia_Sesion_Minima'])
model.Psi_Max = pyo.Param(model.Materias, initialize=datos['Materia_Sesion_Maxima'])
model.gamma_min = pyo.Param(model.Profesores, initialize=datos['Hora_Minima_Profesor'])
model.gamma_max = pyo.Param(model.Profesores, initialize=datos['Hora_Maxima_Profesor'])

### Variables

- $
x^{i, j}_{d, h} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ es asignado la materia } j
    \text{ en la hora } t
    \text{ del dia } d \\
    0 & \text{si no}
\end{cases}
$

- $
y_{i, j} =
\begin{cases}
    1 & \text{si el profesor } i
    \text{ es asignado la materia } j \\
    0 & \text{si no}
\end{cases}
$

- $
z_{j, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ se imparte el dia } d \\
    0 & \text{si no}
\end{cases}
$

- $
s_{j, h, d} =
\begin{cases}
    1 & \text{si la materia } j
    \text{ comienza a la hora } t
    \text{ en el dia } d \\
    0 & \text{si no}
\end{cases}
$

In [61]:
# Lista de variables a gestionar
variables = ['X', 'Y', 'Z', 'S']

# Eliminar variables existentes para evitar sobreescribir
for var in variables:
    if hasattr(model, var):
        model.del_component(getattr(model, var))

# X: Profesor, Materia, Hora, Dia
model.X = pyo.Var(model.Profesores * model.Materias * model.Horas * model.Dias,
                  domain=pyo.Binary)

# Y: Profesor asignado a materia
model.Y = pyo.Var(model.Profesores * model.Materias, domain=pyo.Binary)

# Z: Materia impartida en día d
model.Z = pyo.Var(model.Materias, model.Dias, domain=pyo.Binary)

# S: Materia j en hora h y día d
model.S = pyo.Var(model.Materias, model.Horas, model.Dias, domain=pyo.Binary)

### Restricciones

- No Solapamiento Profesor (No asignar un profesor a más de una materia en un bloque horario):
$$
\sum_{j \in M} x^{i, j}_{t, d} \le 1 \quad \forall\ i \in P,\ t \in T,\ d \in D

In [62]:
#Funciona Ningún profesor puede impartir más de una materia en el mismo bloque horario (Falta checar con mas datos)
if hasattr(model, 'Restriccion_Profesor_Unico'):
    model.del_component(model.Restriccion_Profesor_Unico)

def no_solapamiento_profesor_rule(model, i, t, d):
   return sum(model.X[i, j, t, d]
              for j in model.Materias) <= 1

model.Restriccion_Profesor_Unico = pyo.Constraint(
    model.Profesores, model.Horas, model.Dias,
    rule=no_solapamiento_profesor_rule
)


- El profesor $i$ debe impartir al menos $\gamma_{min}(i)$ horas a la semana y no mas de $\gamma_{max}(i)$ horas a la semana:
$$
\gamma_{min}(i) \leq 
\sum_{j \in M} \sum_{t \in H} \sum_{d \in D} x^{i, j}_{t, d}
\leq \gamma_{max}(i),\ i \in P
$$

In [63]:
# Eliminar la restricción previa si ya existía
if hasattr(model, 'Restriccion_Min_Horas_Profesor'):
    model.del_component(model.Restriccion_Min_Horas_Profesor)

if hasattr(model, 'Restriccion_Max_Horas_Profesor'):
    model.del_component(model.Restriccion_Max_Horas_Profesor)

# def min_horas_profesor_rule(model, i):
#     exprs = [model.X[idx] for idx in X_index if idx[PROFESOR] == i]
#     if not exprs:
#         return pyo.Constraint.Skip
#     return (model.gamma_min[i] <= sum(exprs))
    
def max_horas_profesor_rule(model, i):
    exprs = [
        model.X[i, j, t, d]
        for j in model.Materias
        for t in model.Horas
        for d in model.Dias
    ]
    if not exprs:
        return pyo.Constraint.Skip
    return (sum(exprs) <= model.gamma_max[i])

# model.Restriccion_Min_Horas_Profesor = pyo.Constraint(
#     model.Profesores, rule=min_horas_profesor_rule
# )

model.Restriccion_Max_Horas_Profesor = pyo.Constraint(
    model.Profesores, rule=max_horas_profesor_rule
)

- El curso $j$ debe tener asignado un profesor:
$$
\sum_{i \in P} y_{i, j} \cdot \omega(i, j) = 1,\ \forall\ j \in M
$$

In [64]:
#Funciona Todas las materias tiene profesor
if hasattr(model, 'Restriccion_Materia_Tiene_Profe'):
    model.del_component(model.Restriccion_Materia_Tiene_Profe)

def materia_tiene_profe_rule(model, j):
    return sum(
        model.Y[i, j] * model.omega_prof_materia[i, j]
        for i in model.Profesores
    ) == 1

model.Restriccion_Materia_Tiene_Profe = pyo.Constraint(model.Materias, rule=materia_tiene_profe_rule)

- Todas las horas de la materia $j$ deben estar cubiertas, y el profesor $i$ debe impartir todas las horas del curso $j$, si el es seleccionada para impartirlo:

$$
\sum_{t \in T} \sum_{d \in D} x^{i, j}_{t, d} = \tau(j) \cdot y_{i, j} \quad \forall i \in P,\ j \in M 
$$

In [65]:
#Funciona Se da el numero total de horas semanales requeridas por cada materia
if hasattr(model, 'Restriccion_Profesor_Cubre_Curso'):
    model.del_component(model.Restriccion_Profesor_Cubre_Curso)

def profesor_cubre_curso_rule(model, i, j):
    return sum(
        model.X[i, j, t, d]
        for t in model.Horas
        for d in model.Dias
    ) == model.tau[j] * model.Y[i, j]

model.Restriccion_Profesor_Cubre_Curso = pyo.Constraint(model.Profesores,
                                                        model.Materias,
                                                        rule=profesor_cubre_curso_rule)

- Las sesiones del curso $j$ deben tener una duracion de al menos $\psi_{min}(j)$ hrs y no mas de
$\psi_{max}(j)$ hrs:
$$
\psi_{min}(j) \cdot z_{j, d} \le
\sum_{i \in P} \sum_{t \in T} x^{i, j}_{t, d} \le
\psi_{max}(j) \cdot z_{j, d} \\[0.5em]
j \in M,\ d \in D
$$

In [66]:
if hasattr(model, 'Restriccion_Duracion_Sesion'):
    model.del_component(model.Restriccion_Duracion_Sesion)

if hasattr(model, 'X_jd'):
    model.del_component(model.X_jd)

def min_duracion_rule(model, j, d):
    exprs = [
        model.X[i, j, t, d]
        for i in model.Profesores
        for t in model.Horas
    ]
    if len(exprs) == 0:
        return pyo.Constraint.Skip
    duracion_sesion = sum(exprs)
    return model.Psi_Min[j] * model.Z[j, d] <= duracion_sesion

def max_duracion_rule(model, j, d):
    exprs = [
        model.X[i, j, t, d]
        for i in model.Profesores
        for t in model.Horas
    ]
    if len(exprs) == 0:
        return pyo.Constraint.Skip
    duracion_sesion = sum(exprs)
    return duracion_sesion <= model.Psi_Max[j] * model.Z[j, d]

model.Min_Duracion_Sesion = pyo.Constraint(model.Materias, model.Dias, rule=min_duracion_rule)
model.Max_Duracion_Sesion = pyo.Constraint(model.Materias, model.Dias, rule=max_duracion_rule)

- Solo haya una sesion por dia para una materia $j$ en un dia $d$
$$
z_{j, d} \le 1 \quad j \in M,\ d \in D
$$
$$
\sum_{t \in T} s_{j, t, d} \le z_{j, d} \quad j \in M,\ d \in D
$$

In [67]:
# #No funciona no idea como arreglarla, 

if hasattr(model, 'Restriccion_Sesion_Diaria'):
    model.del_component(model.Restriccion_Sesion_Diaria)

def una_sesion_por_dia_z_rule(model, j, d):
    return model.Z[j, d] <= 1

def una_sesion_por_dia_rule(model, j, d):
    return sum(
        model.S[j, t, d]
        for t in model.Horas
    ) <= model.Z[j, d]

model.Restriccion_Sesion_Diaria_Z = pyo.Constraint(model.Materias,
                                                   model.Dias,
                                                   rule=una_sesion_por_dia_z_rule)

model.Restriccion_Sesion_Diaria = pyo.Constraint(model.Materias,
                                                 model.Dias,
                                                 rule=una_sesion_por_dia_rule)

- Restriccion si una materia inicia en una hora $t$, que la variable $s$ marque su inicio
$$
\alpha(i, j, t, d) = \begin{cases}
    0 & t \le 1 \\
    x_{i, j}^{t, d} & t > 1
\end{cases}
\\[0.5em]

s_{j, t, d} \ge  x^{i, j}_{t, d} - \alpha(i, j, t-1, d) \\[0.5em]
i \in P,\ j \in M,\ t \in T,\ d \in D
$$

In [68]:
if hasattr(model, 'Restriccion_Var_S'):
    model.del_component(model.Restriccion_Var_S)

def var_s_rule(model, i, j, t, d):
    # α(i,j,t-1,d) = 0 if t-1 <= 1 else X[i,j,t-1,d]
    alpha_val = 0 if t <= 1 else model.X[i, j, t-1, d]
    return model.S[j, t, d] >= model.X[i, j, t, d] - alpha_val

model.Restriccion_Var_S = pyo.Constraint(model.Profesores, model.Materias, model.Horas, model.Dias, rule=var_s_rule)

In [69]:
#Modelo V1 Duracion : 30 segundos

# =============================================
# FUNCIÓN OBJETIVO REESCRITA Y OPTIMIZADA
# =============================================

# ------------------------
# Parámetros de ponderación
# ------------------------
M = 100                        # Penalización fuerte para profesores imaginarios

def objetivo_optimo(model):
    # Profesores imaginarios utilizados
    profesores_imaginarios_utilizados = sum(
        model.Y[i, j]
        for i in model.ProfesoresImaginarios
        for j in model.Materias
    )

    # Función objetivo final
    return (
        M * profesores_imaginarios_utilizados
    )

# ------------------------
# Aplicar al modelo
# ------------------------

if hasattr(model, 'Objetivo'):
    model.del_component('Objetivo')

model.Objetivo = pyo.Objective( 
    rule=objetivo_optimo,
    sense=pyo.minimize
)


In [70]:
def resolver_modelo(model):
    from pyomo.opt import SolverFactory, TerminationCondition

    solver = SolverFactory('cplex')

    if not solver.available():
        raise RuntimeError("CPLEX no está disponible. Verifica la instalación o el PATH.")

    resultado = solver.solve(model)

    # Verificar si la solución fue óptima o factible
    condicion = resultado.solver.termination_condition
    if condicion in [TerminationCondition.optimal, TerminationCondition.feasible]:
        print(f"Solución encontrada: {condicion}")
    else:
        print(f"Problema durante la resolución: {condicion}")
        print("Revisa el modelo o ajusta parámetros del solver.")

    return resultado

In [71]:
def mostrar_bloque(dia, hora):
    dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
    dia_str = dias[dia - 1]  # porque 1 = Lunes
    hora_inicio = 7 + hora  # si hora=1 corresponde a 8am
    hora_fin = hora_inicio + 1
    return f"{dia_str} {hora_inicio:02d}:00 - {hora_fin:02d}:00"

def mostrar_asignacion(model):
    asignaciones = []  # lista para guardar los datos

    print("\nAsignaciones de profesores, materias, aulas y horarios:")
    for idx in model.X:
        if pyo.value(model.X[idx]) > 0.5:
            i, j, t, d, = idx
            profesor_nombre = datos['nombres_profesor'].get(i, i)
            materia_nombre = datos['nombres_materia'].get(j, j)
            dia_str = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes'][d - 1]
            hora_inicio = 7 + t
            hora_fin = hora_inicio + 1
            hora_str = f"{hora_inicio:02d}:00 - {hora_fin:02d}:00"
            
            print(f"Profesor, {i}: {profesor_nombre} | Materia, {j}: {materia_nombre} | "
                  f"Horario ({d}, {t}): {dia_str} {hora_str}")
            
            asignaciones.append({
                'Profesor': profesor_nombre,
                'Materia': materia_nombre,
                'Día': dia_str,
                'HoraInicio': f"{hora_inicio:02d}:00",
                'HoraFin': f"{hora_fin:02d}:00",
                'HoraNum': t,  # útil para ordenamientos
                'DiaNum': d,    # útil para ordenamientos
            })

    return asignaciones

In [72]:
# Resolver el modelo
resultados = resolver_modelo(model)

# Verificar si encontró solución
if (resultados.solver.status == pyo.SolverStatus.ok and
    resultados.solver.termination_condition == TerminationCondition.optimal):

    print("Solución óptima encontrada.\n")
    asignaciones_lista = mostrar_asignacion(model)

    import pandas as pd
    df_Una_Asignacion_por_Dia = pd.DataFrame(asignaciones_lista)

else:
    print("No se encontró solución óptima.")
    print("Estado:", resultados.solver.status)
    print("Condición de terminación:", resultados.solver.termination_condition)

Solución encontrada: optimal
Solución óptima encontrada.


Asignaciones de profesores, materias, aulas y horarios:
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 1: Teoría de la Arquitectura | Horario (2, 8): Martes 15:00 - 16:00
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 1: Teoría de la Arquitectura | Horario (2, 9): Martes 16:00 - 17:00
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 1: Teoría de la Arquitectura | Horario (2, 10): Martes 17:00 - 18:00
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 3: Hist. y Tendencias Urbano Arq. | Horario (1, 4): Lunes 11:00 - 12:00
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 3: Hist. y Tendencias Urbano Arq. | Horario (1, 5): Lunes 12:00 - 13:00
Profesor, 100004288: AYALA - ROJAS JUAN LEONARDO | Materia, 3: Hist. y Tendencias Urbano Arq. | Horario (1, 6): Lunes 13:00 - 14:00
Profesor, 100012622: CRUZ - ALMANZA GRACIANO | Materia, 31: Programación Básica | Horario (5, 6): Vierne

In [73]:
df_Una_Asignacion_por_Dia.to_csv("horario.csv")

In [48]:
i = 100012622
j = 31
d = 4
print(pyo.value(model.Z[j, d]))
for t in model.Horas:
    s = model.S[j, t, d]
    x = model.X[i, j, t, d]
    x_prev = 0
    if t > 1:
        x_prev = model.X[i, j, t-1, d]
    output = (f"t: {t} | s = {pyo.value(s)}, x = {pyo.value(x)}, x_prev = {pyo.value(x_prev)}, x - x_prev = {pyo.value(x - x_prev)}")
    print(output)

1.0
t: 1 | s = 0.0, x = 1.0, x_prev = 0, x - x_prev = 1.0
t: 2 | s = 0.0, x = 0.0, x_prev = 1.0, x - x_prev = -1.0
t: 3 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 4 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 5 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 6 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 7 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 8 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 9 | s = 0.0, x = 0.0, x_prev = 0.0, x - x_prev = 0.0
t: 10 | s = 1.0, x = 1.0, x_prev = 0.0, x - x_prev = 1.0


In [ ]:
print()
